# SynthDet Evaluation

In this notebook, we provide the FasterRCNN estimator performance and visualization for predictions on the test split of the [GroceriesReal](https://github.com/Unity-Technologies/SynthDet/blob/master/docs/Readme.md) data. We support performance metrics (mAP, mAP@IOU50, mAR@100) as well as bounding box predictions rendered on the original image. This would provide a better understanding of the given estimator.

<!-- You can use this notebook by the following steps:
- Specify the model path. Then, the notebook would load the checkpoints into a `FasterRCNN` estimator. The `FasterRCNN` can provide model predictions.
- Provide the model performance metrics.
- Can either specify or randomly select some cases for the visualization. -->


## Settings

- Point `data_root` below where you want to download the dataset. If you run this dataset inside a docker container, make sure you point this path to the directory where the external volume is mounted for data storage. 

- Specify a pre-trained `checkpoint_file` that you want to use. You can use one of the estimators that we provided. Optionally, you can use estimators you have trained using our pre-compiled Kubeflow pipeline.

In [ ]:
import os
from pathlib import Path

# Specify data_root
data_root = "/data"
data_path = os.path.join(data_root, "groceries", "v3")

# specify estimator path
# 1. Real-trained estimator on 760 images
# checkpoint_file = "https://storage.googleapis.com/datasetinsights/models/Real-World/FasterRCNN.estimator"

# 2. Synth-trained estimator on 400K SynthDet dataset
# checkpoint_file = "https://storage.googleapis.com/datasetinsights/models/Synthetic/FasterRCNN.estimator"

# 3. Fine-tuned estimator (pre-trained on 400K Synthetic data and fine-tuned on 76 images)
# checkpoint_file = "https://storage.googleapis.com/datasetinsights/models/Synthetic-And-Real-World-76-images/FasterRCNN.estimator"

# 4. Fine-tuned estimator (pre-trained on 400K Synthetic data and fine-tuned on 380 images)
# checkpoint_file = "https://storage.googleapis.com/datasetinsights/models/Synthetic-And-Real-World-380-images/FasterRCNN.estimator"

# 5. Fine-tuned estimator (pre-trained on 400K Synthetic data and fine-tuned on 760 images)
# This is the estimator that provide the best result.
checkpoint_file = "https://storage.googleapis.com/datasetinsights/models/Synthetic-And-Real-World-760-images/FasterRCNN.estimator"

# 6. Your estimator that was stored on GCS
# checkpoint_file = "gs://"

## Download GroceriesReal Dataset

This cell will download the public GroceriesReal dataset to the location specified by data_path. You only need to run this cell once and assuming the dataset exists for the subsequence of the notebook execution. 

The [downloader](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.io.html#datasetinsights.io.loader.create_loader) instantiates the dataset downloader after finding it with the source-uri provided. TODO: will change the dataset name `GroceriesReal` once we have nailed down the name.

In [ ]:
# from datasetinsights.io import create_downloader

# groceries_real_source_uri = "https://storage.googleapis.com/datasetinsights/data/groceries/v3.zip"
# downloader = create_downloader(source_uri=groceries_real_source_uri)
# downloader.download(source_uri=groceries_real_source_uri, output=data_path)

Load GroceriesReal [test](https://github.com/Unity-Technologies/SynthDet/blob/master/docs/Readme.md) dataset. The test dataset has 254 images.

In [ ]:
from datasetinsights.datasets import Dataset

test_dataset = Dataset.create("GroceriesReal", data_path=data_path, split="test")

## Load Estimator
An [estimator](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.estimators.html#datasetinsights.estimators.base.Estimator) is a class of one modeling operation. It includes:

1. input data and output data transformations (e.g. input image cropping, remove unused output labels…) when applicable. 
2. neural network graph (model) for either PyTorch or TensorFlow. 
3. procedures to execute model training and evaluation. <br>

This cell will load an estimator specified in the variable name `checkpoint_file`.

In [ ]:
import tempfile
from yacs.config import CfgNode as CN
from datasetinsights.estimators import create_estimator
    
config_yaml = """
    estimator: FasterRCNN
    backbone: resnet50
    num_classes: 64
    task: object_detection
    test:
      batch_size: 8
      dataset:
        name: GroceriesReal
        args:
          version: v3
          split: test
    metrics:
      mAP:
        name: MeanAveragePrecisionAverageOverIOU
      mAPIOU50:
        name: MeanAveragePrecisionIOU50
      mAR:
        name: MeanAverageRecallAverageOverIOU
    pretrained: False
    pretrained_backbone: True
    synchronize_metrics: True
"""

config = CN.load_cfg(config_yaml)
tmp_dir = tempfile.TemporaryDirectory()
estimator = create_estimator(
    name=config.estimator,
    config=config,
    checkpoint_dir = tmp_dir.name,
    checkpoint_file=checkpoint_file,
    kfp_metrics_dir = tmp_dir.name,
)
tmp_dir.cleanup()
# switch to evaluation mode.
estimator_eval = estimator.model.eval()

## Prediction Visualization

In order to improve visual inspection, we have color-coded bounding boxes predictions based on IOU value between prediction and ground truth bounding boxes. It is considered true positive if `IOU >= 0.5`. We only visualize prediction bounding box with `score >= 0.5`. 

- <font color='green'>Green boxes</font>: If the predicted bounding box can be matched to a ground truth bounding box. <br>
- <font color='red'>Red boxes</font>: If the predicted bounding box can't be matched to a ground truth bounding box. <br>

In [ ]:
from datasetinsights.stats.visualization import match_boxes, plot_bboxes, grid_plot

def visualize_predictions(index=0):
    """ Plot ground truth and prediction for one image.
    
    This method would plot two images: the ground truth is on the left;
    the prediction from the loaded estimator is on the right.
    """
    estimator.model.eval()
    pil_image, gt_bboxes = test_dataset[index]
    pred_bboxes = estimator.predict(pil_image, box_score_thresh=0.5)
    colors = match_boxes(pred_bboxes, gt_bboxes)
    gt_plot = plot_bboxes(pil_image, gt_bboxes, test_dataset.label_mappings)
    pred_plot = plot_bboxes(pil_image, pred_bboxes, test_dataset.label_mappings, colors)
    
    titles = [
        f"ground truth for image index: {index}",
        f"prediction for image index: {index}",
    ]
    grid_plot([[gt_plot, pred_plot]], figsize=(7, 10), img_type="rgb", titles=titles)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

# Please select an index of a image.
interact(visualize_predictions, index=(0, len(test_dataset) - 1))

## Estimator Performance
This section calculates estimator performance on the GroceriesReal Dataset test split. We report the following three evaluation metrics that are commonly used for object detection task:
- [mAP](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.evaluation_metrics.html#datasetinsights.evaluation_metrics.average_precision_2d.MeanAveragePrecisionAverageOverIOU): Average Precision average over all the labels and IOU thresholds = 0.5:0.95:0.05
- [mAPIOU50](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.evaluation_metrics.html#datasetinsights.evaluation_metrics.average_precision_2d.MeanAveragePrecisionIOU50): Mean Average Precision at IOU=50%.
- [mAR](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.evaluation_metrics.html#datasetinsights.evaluation_metrics.average_recall_2d.MeanAverageRecallAverageOverIOU): Average Recall average over all the labels and IOU thresholds = 0.5:0.95:0.05

The next cell will take ~1 hour if you run model precision locally without GPU support. This will make predictions on the whole test set of 254 images. You can reduce computation time significantly on GPU with CUDA support.

In [ ]:
# This is expected to take a while
estimator.evaluate(data_path)

In [ ]:
import pandas as pd

metric_names = ["mAP", "mAPIOU50", "mAR"]
metrics = estimator.metrics
df = pd.DataFrame()
df = pd.DataFrame({
    "Value": [metrics[name].compute() for name in metric_names],
})
df.index = metric_names
df